In [117]:
import warnings
warnings.filterwarnings('ignore')

# Лабораторная работа №5

1) Создали новую таблицу field_employers с двумя атрибутами field_id и professor_id для создания связи "многие ко многим"

2) Проверяем значения таблицы field

In [118]:
import psycopg2 as ps
import pandas as pd

conn = ps.connect(host="localhost", port = 5432, database="orioks", user="postgres", password="123456")

In [119]:
from faker import Faker
import random
from transliterate import translit, get_available_language_codes
import datetime

# Функция для генерации случайных данных для студента
def generate_student_data(group_name):
    fake = Faker('ru_RU')
    gender = random.randint(0, 1)
    student_id = str(random.randint(800000, 999999))  # генерация случайного значения student_id
    if gender == 1:
        surname = fake.last_name_male()
        name = fake.first_name_male()
        patronymic = fake.middle_name_male()
    else:
        surname = fake.last_name_female()
        name = fake.first_name_female()
        patronymic = fake.middle_name_female()
    birthday = fake.date_of_birth(minimum_age=20, maximum_age=21)
    students_group_number = group_name
    email = translit(f"{surname.lower()}.{name.lower()}@miet.ru", 'ru', reversed=True).replace("'", "")
    print(', '.join([student_id, surname, name, patronymic, birthday.strftime('%Y-%m-%d'), students_group_number, email]))

    return student_id, surname, name, patronymic, birthday.strftime('%Y-%m-%d'), students_group_number, email

generate_student_data("РТ-4")

800361, Белозеров, Ефим, Юлианович, 2002-05-23, РТ-4, belozerov.efim@miet.ru


('800361',
 'Белозеров',
 'Ефим',
 'Юлианович',
 '2002-05-23',
 'РТ-4',
 'belozerov.efim@miet.ru')

In [120]:
import random
import string
import psycopg2
from psycopg2 import Error

def create_random_students(connection, num_students, group_name):
    try:
        cursor = connection.cursor()
        #cursor.execute(f"insert into students_group (students_group_number, enrolment_status, structural_unit_number) VALUES ('{group_name}', 'Очная', 1)")

        for _ in range(num_students):
            student_id, surname, name, patronymic, birthday, students_group_number, email = generate_student_data(group_name)
            insert_query = f"INSERT INTO student (student_id, surname, name, patronymic, birthday, students_group_number, email) VALUES ({student_id}, '{surname}', '{name}', '{patronymic}', '{birthday}', '{students_group_number}', '{email}')"
            cursor.execute(insert_query)

        connection.commit()
        print(f"{num_students} new students created successfully.")

    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL:", error)



In [123]:
create_random_students(conn, 25, "РТ-4")

852915, Владимиров, Ермолай, Жанович, 2002-11-06, РТ-4, vladimirov.ermolaj@miet.ru
813998, Аксенов, Фадей, Гордеевич, 2002-02-26, РТ-4, aksenov.fadej@miet.ru
813919, Устинова, Нонна, Болеславовна, 2002-06-27, РТ-4, ustinova.nonna@miet.ru
803306, Агафонова, Евпраксия, Натановна, 2002-09-13, РТ-4, agafonova.evpraksija@miet.ru
960344, Агафонова, Прасковья, Ивановна, 2003-09-12, РТ-4, agafonova.praskovja@miet.ru
934827, Сысоева, Ольга, Оскаровна, 2002-10-31, РТ-4, sysoeva.olga@miet.ru
911528, Фокин, Осип, Германович, 2002-12-30, РТ-4, fokin.osip@miet.ru
994695, Носов, Аникей, Демьянович, 2001-11-28, РТ-4, nosov.anikej@miet.ru
943499, Панова, Кира, Ниловна, 2003-04-03, РТ-4, panova.kira@miet.ru
822561, Трофимов, Леонид, Данилович, 2002-02-20, РТ-4, trofimov.leonid@miet.ru
949179, Елисеева, Нинель, Романовна, 2003-05-17, РТ-4, eliseeva.ninel@miet.ru
952360, Дмитриев, Андрей, Анисимович, 2003-07-29, РТ-4, dmitriev.andrej@miet.ru
804822, Пономарев, Эммануил, Иларионович, 2002-03-01, РТ-4, pono

In [122]:

cursor = conn.cursor()
cursor.execute("DELETE FROM student WHERE students_group_number LIKE '%РТ%'")
conn.commit()

3) Заполняем таблицу field_employers значениями из field

In [18]:
cur = conn.cursor()

cur.execute("SELECT * FROM field_employers;")
if len(cur.fetchall()) < 1:
    cur.execute("insert into field_employers (field_id, professor_id) select field_id, professor_id from field")

Выводим текущие значения в таблице field_employers

In [21]:
-- Active: 1697198327114@@127.0.0.1@5432@orioks@public
SELECT * FROM field_employers LIMIT 15

SyntaxError: illegal target for annotation (4147252753.py, line 1)

![f_e_data](./assets/f_e_data.png)

In [20]:

cur.execute("SELECT * FROM field_employers;")
print('Field Employers\nCount: ',len(cur.fetchall()))
fe = pd.read_sql("SELECT * FROM field_employers LIMIT 15", con=conn)
fe

Field Employers
Count:  60


,field_id,professor_id
0,1fe356aa-5814-ed0d-40b1-4734e13978e9,81001
1,7ace7bee-0d70-5a1f-1113-2cd60d11dfa4,81002
2,fdd39e6d-92ae-7e14-64af-0f023fde62d0,81003
3,bd201d43-4932-4c1d-cd2e-f264ad9c27a5,81004
4,bb8dfd36-8671-d533-8ce3-5afd9a1dc3ca,81005
5,47a8229b-9e8a-0473-fd20-21c889da75bf,81006
6,b075f77e-ed9c-86bf-be5f-98f0ea1ea821,81007
7,2212a60b-1ad4-3eaf-d84b-111286c7511c,81007
8,3af6ad6a-2ca3-81a9-1b6d-ce6ab6525a20,81008
9,c69199ca-fd63-bdc1-3e29-0fc3d67942c7,81009


Добавляем в таблицу professor атрибут телефонный номер

In [ ]:
command = "alter table professor add column tel_num VARCHAR(20) CHECK\
 (tel_num ~ '^(\+7|\+7[0-9])\([0-9]{3}\)[0-9]{3}-[0-9]{2}-[0-9]{2}$'\
     OR\
         tel_num ~ '^8\([0-9]{3}\)[0-9]{3}-[0-9]{2}-[0-9]{2}$');"

cur.execute(command)

Выводим первые 15 значений таблицы

In [ ]:
pf = pd.read_sql("SELECT * FROM professor LIMIT 15", con=conn)
pf

,professor_id,surname,name,patronymic,degree,academic_title,сurrent_position,experience,salary,tel_num
0,81001,Широков,Василий,Иванович,к.т.н.,доцент,доцент,25,"98 000,00 ₽",None
1,81002,Семенов,Андрей,Сергеевич,к.т.н.,профессор,профессор,25,"210 000,00 ₽",None
2,81003,Филатов,Илья,Иванович,к.т.н.,доцент,доцент,13,"85 000,00 ₽",None
3,81004,Ермаков,Виктор,Денисович,None,None,ассистент,1,"12 000,00 ₽",None
4,81005,Марков,Виктор,Дмитриевич,None,None,старший преподаватель,5,"25 000,00 ₽",None
5,81006,Воронов,Николай,Михайлович,к.т.н.,доцент,доцент,25,"98 000,00 ₽",None
6,81007,Хисамов,Василь,Тагирович,None,None,старший преподаватель,5,"25 000,00 ₽",None
7,81008,Козин,Николай,Дмитриевич,None,None,ассистент,1,"12 000,00 ₽",None
8,81009,Петров,Александр,Евгеньевич,None,None,старший преподаватель,10,"55 000,00 ₽",None
9,85001,Березина,Наталия,Владимировна,д.т.н.,профессор,профессор,24,"64 000,00 ₽",None


In [ ]:
command = "alter table students_group add column class_leader_id integer;"

cur.execute(command)

sg = pd.read_sql("SELECT * FROM students_group LIMIT 15", con=conn)
sg

,students_group_number,enrolment_status,structural_unit_number,class_leader_id
0,ИВТ-41,Очная,1,None
1,ИВТ-42,Очная,1,None
2,ИВТ-43,Очная,1,None
3,ИВТ-21В,Заочная,1,None
4,ИБ-21,Очная,3,None
5,ИТД-31,Очная,4,None
6,ИТД-32,Очная,4,None
7,ИТД-33,Очная,4,None


Добавляю проверку для ID старосты группы

In [ ]:
'''
ALTER TABLE students_group
    ADD CONSTRAINT check_group_leader
    FOREIGN KEY (class_leader_id)
    REFERENCES student (student_id);
'''

Новые таблицы для варианта "Студенческий офиса"

![](./assets/my_tables.png)

![db_design](./assets/db_design.png)

Расписание занятий

In [ ]:
'''
CREATE TABLE class_schedule (
    schedule_id SERIAL PRIMARY KEY,
    group_id INT REFERENCES student_groups(group_id),
    subject_id INT REFERENCES subjects(subject_id),
    day_of_week VARCHAR(10),
    start_time TIME,
    end_time TIME,
    teacher_id INT REFERENCES teachers(teacher_id),
    classroom VARCHAR(50)
);
'''

'\nCREATE TABLE class_schedule (\n    schedule_id SERIAL PRIMARY KEY,\n    group_id INT REFERENCES student_groups(group_id),\n    subject_id INT REFERENCES subjects(subject_id),\n    day_of_week VARCHAR(10),\n    start_time TIME,\n    end_time TIME,\n    teacher_id INT REFERENCES teachers(teacher_id),\n    classroom VARCHAR(50)\n);\n'

Заявки на пересдачу

In [ ]:
'''
CREATE TABLE exam_retake_requests (
    request_id SERIAL PRIMARY KEY,
    student_id INT REFERENCES students(student_id),
    subject_id INT REFERENCES subjects(subject_id),
    reason VARCHAR(255),
    status VARCHAR(20), -- In progress, Approved, Rejected
    request_date DATE
);
'''

'\nCREATE TABLE exam_retake_requests (\n    request_id SERIAL PRIMARY KEY,\n    student_id INT REFERENCES students(student_id),\n    subject_id INT REFERENCES subjects(subject_id),\n    reason VARCHAR(255),\n    status VARCHAR(20), -- In progress, Approved, Rejected\n    request_date DATE\n);\n'

Академический календарь

In [ ]:
'''
CREATE TABLE academic_calendar (
    event_id SERIAL PRIMARY KEY,
    event_name VARCHAR(100),
    start_date DATE,
    end_date DATE,
    event_type VARCHAR(50) -- Semester, Holidays, Exams, etc.
);
'''

'\nCREATE TABLE academic_calendar (\n    event_id SERIAL PRIMARY KEY,\n    event_name VARCHAR(100),\n    start_date DATE,\n    end_date DATE,\n    event_type VARCHAR(50) -- Semester, Holidays, Exams, etc.\n);\n'